In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates

In [2]:
client = MongoClient()
db = client.obamacare

## Levantar grafos separados

In [3]:
path = "../"
df = pd.read_csv(path+'USER_POLARITY_BARBERA.txt',sep='\t',header=None).dropna(axis=1)
df.columns = ['user_id','leaning']
df = df.set_index('user_id')
user_leaning = df.to_dict()['leaning']

In [4]:
def get_users_tweets_full(user_leaning):
    user_tweets = {}
    tweets = []
    min_date = datetime(3000,1,1) 
    max_date = datetime(1900,1,1)
    users_tweets = {}
    step = 10000
    total = db.tweetsInfo.count_documents({'userId': {'$in': list(user_leaning.keys())}})
    for t in tqdm(db.tweetsInfo.find({'userId': {'$in': list(user_leaning.keys())}}), total=total):
        tweets.append(t['tweetId'])
        user_tweets.setdefault(t['userId'], list()).append(t['tweetId'])
        created = t['created']
        if min_date > created:
            min_date = created
        if max_date < created:
            max_date = created
    return set(user_tweets.keys()), user_tweets, tweets, min_date, max_date



def get_users_tweets(users, conditions={}):
    users_tweets = {}
    cond = {'userId': {'$in': list(users)}}
    cond.update(conditions)
    total = db.tweetsInfo.count_documents(cond)
    for t in tqdm(db.tweetsInfo.find(cond), total=total):
        users_tweets.setdefault(t['userId'], list()).append(t['tweetId'])
    return users_tweets


def add_created(link, date, link_date):
    if link not in link_date:
        link_date[link] = date
        return
    if link_date[link] > date:
        link_date[link] = date
    pass

def users_retweeted(tweet, conditions={}):
    if 'retweetId' not in tweet:
        return set()
    cond = {'tweetId': tweet['retweetId']}
    cond.update(conditions)
    n_tweet = db.tweetsInfo.find_one(cond)
    if n_tweet is None:
        return set()
    return {n_tweet['userId']}
    
    
def users_replies_from_tweet(userId, tweets, recursive, user_set, conditions={}):
    users = Counter()
    replies_date = dict()
    retweets_tweets = dict()
    cond = {'userId': userId}
    cond.update(conditions)
    graph = {
        'startWith': '$inReplyToStatusId',
        'from': 'tweetsInfo',
        'connectFromField': 'inReplyToStatusId',
        'connectToField': 'tweetId',
        'as': 'replies'
    }
    if not recursive:
        graph['maxDepth'] = 0
    cursor = db.tweetsInfo.aggregate([{'$match': cond},
                                  {'$graphLookup': graph},
                                  {'$project': {'tweetId': 1, 
                                                'created': 1, 
                                                'replies.tweetId': 1, 
                                                'replies.userId': 1}}])
    cursor = list(cursor)
    cursor.sort(key=lambda x: x['created'], reverse=True)
    processed = set()
    for x in cursor:
        if x['tweetId'] not in tweets:
            continue
        userSet = set()
        for r in x['replies']:
            if r['userId'] not in user_set:
                processed.add(r['tweetId'])
                continue
            add_created((userId, r['userId']), x['created'], replies_date)
            if r['tweetId'] not in processed:
                userSet.add(r['userId'])
                #Si ya hay un link lo actualizo
                if (userId, r['userId']) in retweets_tweets:
                    #Verifico si es por la misma cadena de tweets
                    base = retweets_tweets[(userId, r['userId'])]
                    if x['tweetId'] in base:
                        base[x['tweetId']].append(r['tweetId'])
                    else:
                        base[x['tweetId']] = [r['tweetId']]
                else:
                    #Agrego el tweet a mi lista de bases. El link no existia hasta ahora
                    retweets_tweets[(userId, r['userId'])] = {x['tweetId']: [r['tweetId']]}
                processed.add(r['tweetId'])
        if userId in userSet:
            userSet.remove(userId)
        for u in userSet:
            users[(userId, u)] += 1
    if (userId, userId) in replies_date:
        replies_date.pop((userId, userId))
        retweets_tweets.pop((userId, userId))
    return users, replies_date, retweets_tweets

    
def calculate_graphs(user_tweets, recursive=True, conditions={}):
    replies = Counter()
    mentions = Counter()
    retweets = Counter()
    replies_date = dict()
    mentions_date = dict()
    retweets_date = dict()
    replies_tweets = dict()
    mentions_tweets = dict()
    retweets_tweets = dict()
    '''tweet_user = {}
    for u, tweets in user_tweets.items():
        for t in tweets:
            tweet_user[t] = u'''
    user_set = set(user_tweets)
    #nus = 0
    for user, tweets in tqdm(user_tweets.items()):
        #nus += 1
        #if nus > 1660:
        #    print(user, tweets)
        #Mentions
        cond = {'tweetId': {'$in': tweets}}
        cond.update(conditions)
        for tweet in db.tweetsInfo.find(cond):
            #Mentions
            for mention in tweet['userMentions']:
                m_user = mention['userId']
                if user != m_user and m_user in user_set:
                    mentions[(user, m_user)] += 1
                    add_created((user, m_user), tweet['created'], mentions_date)
                    if (user, m_user) in mentions_tweets:
                        mentions_tweets[(user, m_user)].append(tweet['tweetId'])
                    else:
                        mentions_tweets[(user, m_user)] = [tweet['tweetId']]
            #Retweet
            if 'retweetId' not in tweet:
                continue
            for m_user in users_retweeted(tweet, conditions):
                if user != m_user and m_user in user_set:
                    retweets[(user, m_user)] += 1
                    add_created((user, m_user), tweet['created'], retweets_date)
                    if (user, m_user) in retweets_tweets:
                        retweets_tweets[(user, m_user)].append(tweet['tweetId'])
                    else:
                        retweets_tweets[(user, m_user)] = [tweet['tweetId']]
        #Replies
        u_replies, u_replies_date, u_replies_tweets = users_replies_from_tweet(user, set(tweets), recursive, user_set, conditions)
        replies += u_replies
        replies_date.update(u_replies_date)
        replies_tweets.update(u_replies_tweets)
    full_users = {u: True for u in user_tweets.keys()}
    for u in (replies + mentions + retweets).keys():
        if u[0] not in full_users:
            full_users[u[0]] = False
        if u[1] not in full_users:
            full_users[u[1]] = False
    return full_users, replies, mentions, retweets, replies_date, mentions_date, retweets_date, replies_tweets, mentions_tweets, retweets_tweets

def create_simple_graph(users, links, dates, tweets):
    graph = nx.DiGraph()
    for u, c in users.items():
        graph.add_node(u, central=c)
    for (org, dest), weight in links.items():
        graph.add_edge(org, dest, weight=weight, date=dates[(org, dest)], tweets=tweets[(org, dest)])
    return graph

def create_full_graph(users, replies, mentions, retweets,\
                      replies_date, mentions_date, retweets_date,\
                      replies_tweets, mentions_tweets, retweets_tweets):
    graph = nx.DiGraph()
    for u, c in users.items():
        graph.add_node(u, central=c)
    links = set(replies.keys()) | set(mentions.keys()) | set(retweets.keys())
    for l in links:
        graph.add_edge(l[0], l[1], weight=replies.get(l, 0) + mentions.get(l, 0) + retweets.get(l, 0), \
                       date=min([f[l] for f in [replies_date, mentions_date, retweets_date] if l in f]), \
                       date_replies=replies_date.get(l, None), \
                       date_mentions=mentions_date.get(l, None), \
                       date_retweets=retweets_date.get(l, None), \
                       tweets_replies=replies_tweets.get(l, []), \
                       tweets_mentions=mentions_tweets.get(l, []), \
                       tweets_retweets=retweets_tweets.get(l, [])
                      )
    return graph


In [5]:
def get_graphs_training_test(user_leaning, date=datetime(2017, 8, 30)):
    users, user_tweets, tweets, min_date, max_date = get_users_tweets_full(user_leaning)
    full_users = {u: True for u  in users}
    print('Training')
    cond = {'created': {'$gte': min_date, '$lte': date}}
    user_tweets = get_users_tweets(users, conditions=cond)
    _, replies, mentions, retweets, replies_date, mentions_date, retweets_date, replies_tweets, mentions_tweets, retweets_tweets = calculate_graphs(user_tweets, conditions=cond)
    mentions_graph_train = create_simple_graph(full_users, mentions, mentions_date, mentions_tweets)
    replies_graph_train = create_simple_graph(full_users, replies, replies_date, replies_tweets)
    retweets_graph_train = create_simple_graph(full_users, retweets, retweets_date, retweets_tweets)
    graph_train = create_full_graph(full_users, replies, mentions, retweets, replies_date, mentions_date, retweets_date, replies_tweets, mentions_tweets, retweets_tweets)
    print('test')
    cond = {'created': {'$gt': date, '$lte': max_date}}
    user_tweets = get_users_tweets(users, conditions=cond)
    _, replies, mentions, retweets, replies_date, mentions_date, retweets_date, replies_tweets, mentions_tweets, retweets_tweets = calculate_graphs(user_tweets, conditions=cond)
    mentions_graph_test = create_simple_graph(full_users, mentions, mentions_date, mentions_tweets)
    replies_graph_test = create_simple_graph(full_users, replies, replies_date, replies_tweets)
    retweets_graph_test = create_simple_graph(full_users, retweets, retweets_date, retweets_tweets)
    graph_test = create_full_graph(full_users, replies, mentions, retweets, replies_date, mentions_date, retweets_date, replies_tweets, mentions_tweets, retweets_tweets)

    return (graph_train, mentions_graph_train, replies_graph_train, retweets_graph_train), (graph_test, mentions_graph_test, replies_graph_test, retweets_graph_test)


(graph_train, mentions_graph_train, replies_graph_train, retweets_graph_train), \
(graph_test, mentions_graph_test, replies_graph_test, retweets_graph_test) = get_graphs_training_test(user_leaning)

  0%|          | 0/7017509 [00:00<?, ?it/s]

Training


  0%|          | 0/5208255 [00:00<?, ?it/s]

  0%|          | 0/6352 [00:00<?, ?it/s]

test


  0%|          | 0/1809254 [00:00<?, ?it/s]

  0%|          | 0/4941 [00:00<?, ?it/s]

In [6]:
os.makedirs('train')
nx.write_gpickle(graph_train, 'train/full.gpickle')
nx.write_gpickle(mentions_graph_train, 'train/mentions.gpickle')
nx.write_gpickle(replies_graph_train, 'train/replies.gpickle')
nx.write_gpickle(retweets_graph_train, 'train/retweets.gpickle')
os.makedirs('test')
nx.write_gpickle(graph_test, 'test/full.gpickle')
nx.write_gpickle(mentions_graph_test, 'test/mentions.gpickle')
nx.write_gpickle(replies_graph_test, 'test/replies.gpickle')
nx.write_gpickle(retweets_graph_test, 'test/retweets.gpickle')

In [8]:
#Train target
user_central = {u for u in graph_train.nodes if graph_train.nodes[u]['central']}
cant = 0
for n in user_central:
    neighbors = graph_train.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1

print(cant)

311611


In [9]:
#Test target
user_central = {u for u in graph_test.nodes if graph_train.nodes[u]['central']}
cant = 0
for n in user_central:
    neighbors = graph_test.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1

print(cant)

125218


In [10]:
'''
user_central = {u for u in graph_train.nodes if graph.nodes[u]['central']}
org_dest = set()
for n in user_central:
    neighbors = graph_train.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            org_dest.add(n)

reachables = set()

for n in org_dest:
    neighbors = graph_train.neighbors(n)
    reachables.update(neighbors)

unreachables = set(graph_train.nodes) - (org_dest | reachables)
graph_train.remove_nodes_from(unreachables)
mentions_graph_train.remove_nodes_from(unreachables)
replies_graph_train.remove_nodes_from(unreachables)
retweets_graph_train.remove_nodes_from(unreachables)
graph_test.remove_nodes_from(unreachables)
mentions_graph_test.remove_nodes_from(unreachables)
replies_graph_test.remove_nodes_from(unreachables)
retweets_graph_test.remove_nodes_from(unreachables)
'''

"\nuser_central = {u for u in graph_train.nodes if graph.nodes[u]['central']}\norg_dest = set()\nfor n in user_central:\n    neighbors = graph_train.neighbors(n)\n    for neo in neighbors:\n        if neo in user_central:\n            org_dest.add(n)\n\nreachables = set()\n\nfor n in org_dest:\n    neighbors = graph_train.neighbors(n)\n    reachables.update(neighbors)\n\nunreachables = set(graph_train.nodes) - (org_dest | reachables)\ngraph_train.remove_nodes_from(unreachables)\nmentions_graph_train.remove_nodes_from(unreachables)\nreplies_graph_train.remove_nodes_from(unreachables)\nretweets_graph_train.remove_nodes_from(unreachables)\ngraph_test.remove_nodes_from(unreachables)\nmentions_graph_test.remove_nodes_from(unreachables)\nreplies_graph_test.remove_nodes_from(unreachables)\nretweets_graph_test.remove_nodes_from(unreachables)\n"

In [11]:
user_central = {u for u in graph_train.nodes if graph_train.nodes[u]['central']}
cant = 0
for n in user_central:
    neighbors = graph_train.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1

print(cant)

cant = 0
for n in user_central:
    neighbors = graph_test.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1

print(cant)

cant = 0
for n in user_central:
    neighbors = graph_train.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1
            break

print(cant)

cant = 0
for n in user_central:
    neighbors = graph_test.neighbors(n)
    for neo in neighbors:
        if neo in user_central:
            cant += 1
            break

print(cant)

311611
125218
5399
3652


## Embeddings

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Add, Lambda, Concatenate
from tensorflow.keras.models import Model
import functools
import tensorflow.keras.backend as K
from scipy.sparse import dok_matrix
import random


seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

In [30]:
user_central = [u for u in graph_train.nodes if graph_train.nodes[u]['central']]
user_central.sort()
id_users = list(graph_train.nodes)

def cmp(x, y):
    if graph_train.nodes[x]['central'] and not graph_train.nodes[y]['central']:
        return -1
    if not graph_train.nodes[x]['central'] and graph_train.nodes[y]['central']:
        return 1
    return x - y

id_users.sort(key=functools.cmp_to_key(cmp))
user_id = {u: e for e, u in enumerate(id_users)}

In [31]:
x_target = []
x_context = []

y = []

for edge in graph_train.edges:
    x_target.append(user_id[edge[0]])
    x_context.append(user_id[edge[1]])
    x_target.append(user_id[edge[1]])
    x_context.append(user_id[edge[0]])
    y.append(graph_train.edges[edge]['weight'] + 1)
    y.append(graph_train.edges[edge]['weight'] + 1)

In [32]:
x_target = np.asarray(x_target, dtype=np.float32)
x_target = np.expand_dims(x_target, axis=-1)
x_context = np.asarray(x_context, dtype=np.float32)
x_context = np.expand_dims(x_context, axis=-1)
y = np.asarray(y, dtype=np.float32)
y = np.expand_dims(y, axis=-1)

In [33]:
def custom_loss(y_true, y_pred, a = 3.0/4.0, X_MAX=100):
    """
    This is GloVe's loss function
    :param y_true: The actual values, in our case the 'observed' X_ij co-occurrence values
    :param y_pred: The predicted (log-)co-occurrences from the model
    :return: The loss associated with this batch
    """
    return K.sum(K.pow(K.clip(y_true / X_MAX, 0.0, 1.0), a) * K.square(y_pred - K.log(y_true)), axis=-1)


def glove_model(vocab_size=10, vector_dim=64):
    """
    A Keras implementation of the GloVe architecture
    :param vocab_size: The number of distinct words
    :param vector_dim: The vector dimension of each word
    :return:
    """
    input_target = Input((1,), name='central_word_id')
    input_context = Input((1,), name='context_word_id')

    central_embedding = Embedding(vocab_size+1, vector_dim, input_length=1, name='central_emb')
    central_bias = Embedding(vocab_size+1, 1, input_length=1, name='central_bias')

    context_embedding = Embedding(vocab_size, vector_dim, input_length=1, name='context_emb')
    context_bias = Embedding(vocab_size, 1, input_length=1, name='context_bias')

    vector_target = central_embedding(input_target)
    vector_context = context_embedding(input_context)

    bias_target = central_bias(input_target)
    bias_context = context_bias(input_context)

    dot_product = Dot(axes=-1)([vector_target, vector_context])
    dot_product = Reshape((1, ))(dot_product)
    bias_target = Reshape((1,))(bias_target)
    bias_context = Reshape((1,))(bias_context)

    prediction = Add()([dot_product, bias_target, bias_context])

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss=custom_loss, optimizer='adam')

    return model


def glove_model(vocab_size=10, vector_dim=64):
    """
    A Keras implementation of the GloVe architecture
    :param vocab_size: The number of distinct words
    :param vector_dim: The vector dimension of each word
    :return:
    """
    input_target = Input((1,), name='central_word_id')
    input_context = Input((1,), name='context_word_id')

    central_embedding = Embedding(vocab_size+1, vector_dim, input_length=1, name='central_emb')

    vector_target = central_embedding(input_target)
    #vector_target = Lambda(lambda x: K.abs(x))(vector_target)
    vector_context = central_embedding(input_context)
    #vector_context = Lambda(lambda x: K.abs(x))(vector_context)

    prediction = Dot(axes=-1)([vector_target, vector_context])

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss=custom_loss, optimizer='adam')
    
    model.summary()

    return model

In [34]:
from tensorflow.keras.utils import Sequence
import math 


class GloveNegExample(Sequence):
    
    def __init__(self, x_target, x_content, y, users, batch_size=128, neg_sampling=1):
        self.x_target = x_target
        self.x_content = x_content
        self.y = y
        self.users = users
        self.batch_size = batch_size
        self.neg_sampling = 1
        self.idx = list(range(self.y.shape[0]))
        random.shuffle(self.idx)
        pass
        
    def __len__(self):
        return math.ceil(len(self.idx) / self.batch_size)
    
    def __getitem__(self, idx):
        x_content = self.x_content[(idx * self.batch_size) : ((idx + 1) * self.batch_size), ...]
        x_target = self.x_target[(idx * self.batch_size) : ((idx + 1) * self.batch_size), ...]
        y = self.y[(idx * self.batch_size) : ((idx + 1) * self.batch_size), ...]
        neg_size = int(y.shape[0] * self.neg_sampling)
        x_content_neg = np.asarray([[random.randrange(self.users)] for _ in range(neg_size)])
        x_target_neg = np.asarray([[random.randrange(self.users)] for _ in range(neg_size)])
        y_neg = np.asarray([[1]] * neg_size)
        return [np.concatenate((x_content, x_content_neg), axis=0), \
                np.concatenate((x_target, x_target_neg), axis=0)], \
                np.concatenate((y, y_neg), axis=0)
    
    def on_epoch_end(self):
        random.shuffle(self.idx)
        pass

In [35]:
model = glove_model(len(id_users), 10)

if not os.path.exists('embeddedings-dates.h5'):
    model.fit(GloveNegExample(x_context, x_target, y, len(user_id)), epochs=15) 
    model.save_weights('embeddedings-dates.h5')
else:
    model.load_weights('embeddedings-dates.h5')

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
central_word_id (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
context_word_id (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
central_emb (Embedding)         (None, 1, 10)        64510       central_word_id[0][0]            
                                                                 context_word_id[0][0]            
__________________________________________________________________________________________________
dot_3 (Dot)                     (None, 1, 1)         0           central_emb[0][0]     

In [36]:
embeddings = K.get_value(model.layers[2].embeddings)

# Commuties

In [37]:
def to_undirected_graph(graph):
    g = nx.Graph()
    for node in graph.nodes:
        g.add_node(node)
    added = set()
    for l in graph.edges:
        if l not in added:
            g.add_edge(*l)
            added.add(l)
            added.add(l[::-1])
    return g

In [38]:
from networkx.algorithms.community import greedy_modularity_communities
import json

if not os.path.exists('commutities_v2.json'):
    com = list(greedy_modularity_communities(to_undirected_graph(graph_train)))
    with open('commutities_v2.json', 'w') as outfile:
        json.dump([list(x) for x in com], outfile)
else:
    with open('commutities_v2.json', 'r') as json_file:
        com = [set(x) for x in json.load(json_file)]

In [39]:
user_com = {}
for c, group in enumerate(com):
    for u in group:
        user_com[u] = c

In [40]:
if not os.path.exists('cos.npz'):
    users_embs = embeddings[0:len(user_central), :]

    cos = np.dot(users_embs, users_embs.T)
    len_emb = np.expand_dims(np.sum(users_embs ** 2, axis=1) ** 0.5, axis=-1)
    cos = cos / np.dot(len_emb, len_emb.T)
    np.savez_compressed('cos.npz', cosines=cos)
else:
    cos = np.load('cos.npz')['cosines']
del embeddings

In [41]:
from collections import deque
sim = deque()
diff = deque()
all_cos = deque()
for i in tqdm(range(cos.shape[0])):
    for j in range(i+1, cos.shape[1]):
        all_cos.append(cos[i, j])
        if user_com[id_users[i]] == user_com[id_users[j]]:
            sim.append(cos[i, j])
        else:
            diff.append(cos[i, j])

  0%|          | 0/6450 [00:00<?, ?it/s]

In [42]:
print(len(all_cos))
print(np.mean(all_cos))
print(np.std(all_cos))
print(len(sim))
print(np.average(sim))
print(np.std(sim))
print(len(diff))
print(np.average(diff))
print(np.std(diff))

20798025
0.30522174
0.33704013
9623408
0.37413183
0.34416634
11174617
0.24587648
0.3190679


In [43]:
from scipy.stats import ttest_ind

ttest_ind(sim, diff, alternative='greater', equal_var=False)

Ttest_indResult(statistic=876.3486836895261, pvalue=0.0)

In [44]:
ttest_ind(sim, all_cos, alternative='greater', equal_var=False)

Ttest_indResult(statistic=516.9322873042893, pvalue=0.0)

In [45]:
ttest_ind(all_cos, diff, alternative='greater', equal_var=False)

Ttest_indResult(statistic=491.61336296710385, pvalue=0.0)

In [46]:
cos[:5, :5]

array([[0.99999994, 0.4030889 , 0.3625932 , 0.17443882, 0.4852578 ],
       [0.4030889 , 1.        , 0.6120374 , 0.6151954 , 0.49913073],
       [0.3625932 , 0.6120374 , 0.9999999 , 0.55664915, 0.7531488 ],
       [0.17443882, 0.6151954 , 0.55664915, 1.        , 0.68268776],
       [0.4852578 , 0.49913073, 0.7531488 , 0.68268776, 1.0000001 ]],
      dtype=float32)

In [47]:
del sim
del diff
del all_cos

In [48]:
nx.write_gpickle(graph_train, 'graph_train.gpickle')
nx.write_gpickle(mentions_graph_train, 'mentions_graph_train.gpickle')
nx.write_gpickle(replies_graph_train, 'replies_graph_train.gpickle')
nx.write_gpickle(retweets_graph_train, 'retweets_graph_train.gpickle')
nx.write_gpickle(graph_test, 'graph_test.gpickle')
nx.write_gpickle(mentions_graph_test, 'mentions_graph_test.gpickle')
nx.write_gpickle(replies_graph_test, 'replies_graph_test.gpickle')
nx.write_gpickle(retweets_graph_test, 'retweets_graph_test.gpickle')